In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-generated-multi-data/subtaskA_train_multilingual.jsonl
/kaggle/input/llm-generated-multi-data/subtaskA_dev_multilingual.jsonl
/kaggle/input/llms-generated-text-mono/subtaskA_dev_monolingual.jsonl
/kaggle/input/llms-generated-text-mono/subtaskA_train_monolingual.jsonl


# Necessary libraries

In [2]:
# Libraries
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import pandas as pd 

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Handling Raw Corpus

In [3]:
import json
# Read a jsonl file into list-of-dictionaries
def read_jsonl(filename):
    pairs = []
    for line in open(filename, 'r', encoding='utf-8'):
            pairs.append(json.loads(line))
    if len(pairs) == 1:
        pairs = pairs[0]
    print("\n# Samples: ", len(pairs))
    print("# Keys: ", pairs[0].keys())
    return pairs

In [4]:
# load the dataset
import pandas as pd
from tqdm import tqdm
train_data_path = "/kaggle/input/llm-generated-multi-data/subtaskA_train_multilingual.jsonl"
train_data = read_jsonl(train_data_path)
all_mono_train_contents = []
all_mono_valid_contents = []
for i in tqdm(range(len(train_data))):
    text = train_data[i]['text']
    label = train_data[i]['label']
    all_mono_train_contents.append(text)
    all_mono_valid_contents.append(label)
# dictionary of lists 
dict = {'text': all_mono_train_contents, 'label': all_mono_valid_contents} 
trainDF = pd.DataFrame(dict)
# trainDF = pd.read_csv('data.csv', delimiter='\t')
# #trainDF = pd.read_csv('/content/drive/MyDrive/Question Classifier Telugu/Finalized_samples.csv', delimiter=',')
# # trainDF = pd.read_csv('/content/drive/MyDrive/Question Classifier Telugu/abs_eval.csv', delimiter=',')

# trainDF.head()


# Samples:  172417
# Keys:  dict_keys(['text', 'label', 'model', 'source', 'id'])


100%|██████████| 172417/172417 [00:00<00:00, 710510.85it/s]


In [5]:
dev_data_path = "/kaggle/input/llm-generated-multi-data/subtaskA_dev_multilingual.jsonl"
dev_data = read_jsonl(dev_data_path)
all_mono_dev_text_contents = []
all_mono_dev_label_contents = []
for i in tqdm(range(len(dev_data))):
    text = dev_data[i]['text']
    label = dev_data[i]['label']
    all_mono_dev_text_contents.append(text)
    all_mono_dev_label_contents.append(label)
# dictionary of lists 
dict = {'text': all_mono_dev_text_contents, 'label': all_mono_dev_label_contents} 
testDF = pd.DataFrame(dict)


# Samples:  4000
# Keys:  dict_keys(['text', 'label', 'model', 'source', 'id'])


100%|██████████| 4000/4000 [00:00<00:00, 677839.93it/s]


In [6]:
testDF['label'][0]

1

## Data Splitting into Train and Test

In [ ]:
# # split the dataset into training and validation datasets 
# train_x, test_x, train_y, test_y = model_selection.train_test_split(trainDF['text'], trainDF['category'], test_size=0.2, random_state=42)
# #print(train_y.value_counts())
# # label encode the target variable 
# encoder = preprocessing.LabelEncoder()
# train_y = encoder.fit_transform(train_y)
# test_y = encoder.fit_transform(test_y)
# print(test_y)
# classes = list(encoder.classes_)
# print(len(classes))
# print(train_y)

In [7]:
from sklearn import model_selection, preprocessing

# Assuming trainDF and testDF are your existing DataFrames
# You can directly use trainDF and testDF without splitting

# Use the 'text' and 'category' columns from trainDF and testDF
train_x = trainDF['text']
train_y = trainDF['label']

test_x = testDF['text']
test_y = testDF['label']

# Label encode the target variable
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)  # Use transform instead of fit_transform for the test set

# Now train_y and test_y are the encoded labels
print(test_y)

# Get the class labels
classes = list(encoder.classes_)
print(len(classes))

# Print the encoded training labels
print(train_y)

[1 1 1 ... 1 1 1]
2
[1 1 1 ... 1 1 1]


# Feature Engineering: Counter Vectorizer

In [8]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])
# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.fit_transform(train_x)
#print(xtrain_count)
xvalid_count =  count_vect.transform(test_x)
#print(xvalid_count)
xtrain_count.shape, xvalid_count.shape

((172417, 990912), (4000, 990912))

# Feature Engineering: TF-IDF Vectorizer

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern = r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(test_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', ngram_range=(2,3), max_features=25000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(test_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', ngram_range=(2,3), max_features=25000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(test_x) 

# Model Traning

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    if is_neural_net:
        predictions = predictions.argmax(axis=-1)

    
    acc = metrics.accuracy_score(predictions, test_y)
    f1 = metrics.f1_score(predictions, test_y, average='weighted')
    #print(classification_report(predictions, test_y, target_names = list(encoder.classes_)))
    return acc, f1



## Logistic Regression

In [ ]:
# # Linear Classifier on Count Vectors
# accuracy, f1_score = train_model(linear_model.LogisticRegression(solver='sag',multi_class='multinomial', max_iter=25000), xtrain_count, train_y, xvalid_count)
# print("LR, Count Vectors: ", accuracy, f1_score)

# # Linear Classifier on Word Level TF IDF Vectors
# accuracy, f1_score = train_model(linear_model.LogisticRegression(multi_class='multinomial', solver='sag', max_iter=25000), xtrain_tfidf, train_y, xvalid_tfidf)
# print("LR, WordLevel TF-IDF: ", accuracy, f1_score)

# # Linear Classifier on Ngram Level TF IDF Vectors
# accuracy, f1_score = train_model(linear_model.LogisticRegression(multi_class='multinomial', solver='sag', max_iter=25000), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print("LR, N-Gram Vectors: ", accuracy, f1_score)

# # Linear Classifier on Character Level TF IDF Vectors
# accuracy, f1_score = train_model(linear_model.LogisticRegression(multi_class='multinomial', solver='sag', max_iter=25000), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
# print("LR, CharLevel Vectors: ", accuracy, f1_score)

## Naive Bayes Classifier

In [ ]:
# # Naive Bayes on Count Vectors
# accuracy, f1_score = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
# print("NB, Count Vectors: ", accuracy, f1_score)


# # Naive Bayes on Word Level TF IDF Vectors
# accuracy, f1_score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
# print("NB, WordLevel TF-IDF: ", accuracy, f1_score)

# # Naive Bayes on Ngram Level TF IDF Vectors
# accuracy, f1_score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print("NB, N-Gram Vectors: ", accuracy, f1_score)

# # Naive Bayes on Character Level TF IDF Vectors
# accuracy, f1_score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
# print("NB, CharLevel Vectors: ", accuracy, f1_score)

## Support Vector Machine

In [ ]:
# # SVM on Count Vectors
# accuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
# print("SVM, Count Vectors: ", accuracy)

# SVM on TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print("SVM, TF-IDF Vectors: ", accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, Ngram Level Vectors: ", accuracy)

# SVM on Character Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("SVM, Character Level Vectors: ", accuracy)

## Random Forest Classifier

In [ ]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=10, min_samples_split=2, n_jobs=-1), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy)


# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=10, min_samples_split=2, n_jobs=-1), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("RF, NgramLevel TF-IDF: ", accuracy)


# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=10, min_samples_split=2, n_jobs=-1), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("RF, Character Level TF-IDF: ", accuracy)

## Boosting Model

In [ ]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy)

## MLP Classifier

In [ ]:
# MLP with Counter Vectors
accuracy = train_model(MLPClassifier(random_state=1, max_iter=300), xtrain_count, train_y, xvalid_count)
print("MLP, Counter Vectors: ", accuracy)

# MLP with TF-IDF Word Level Vectors
accuracy = train_model(MLPClassifier(random_state=1, max_iter=300), xtrain_tfidf, train_y, xvalid_tfidf)
print("MLP, TF-IDF Word Level: ", accuracy)

# MLP with TF-IDF N-gram Level Vectors
accuracy = train_model(MLPClassifier(random_state=1, max_iter=300), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("MLP, TF-IDF Ngram Level: ", accuracy)

# MLP with TF-IDF Character Level Vectors
accuracy = train_model(MLPClassifier(random_state=1, max_iter=300), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("MLP, TF-IDF Character Level: ", accuracy)